In [2]:
import pecos as pc
import numpy as np
import itertools
from bitstring import BitArray

In [3]:
pc.__version__

'0.2.dev'

**Task**: Verify fault-tolerance of the $|0_L\rangle$-preparation circuit with flag qubit.
* The Stabilizer generators of the Steane code are pure (don't mix X/Z): <IIIZZZZ,ZIZIZIZ,IZZIIZZ> + <IIIXXXX,XIXIXIX,IXXIIXX>.
* Thus, we can do the stabilizer measurements and error corrections in series, first for expl for Z, then for X. (Not implemented here.)
* This circuit **prepares** the state $|0_L\rangle$ of the Steane code (it neither measures syndromes nor corrects any errors).
* The $|0_L\rangle$ state is not unique. Every +1-eigenstate of a Z-stabilizer is a $|0_L\rangle$ state, as $Z_L|0_L\rangle=+1|0_L\rangle$ for $Z_L=ZZZZZZZ$ and since the stabilizers are linearly independent: $[S_i,S_j]=0$
* Furthermore, every -1-eigenstate of a Z-stabilizer can be found by bitwise-inversion: $X_L |0_L\rangle=|1_L\rangle$ with $X_L=XXXXXXX$, expl. `0b0001111` ^ `0b1111111` = `0b1110000`, s.t. $Z_L|1_L\rangle=-1|1_L\rangle$.
* **Note** that both $|0_L\rangle$ and $|1_L\rangle$ are +1 eigenstates of the stabilizers, but only correspond to $\pm1$-eigenstates of the logical operator $Z_L$.
* Thus, this circuit guarantees that for any possible single- or 2-qubit fault in the circuit, the output state will be one of the $|0_L\rangle$ states with max. 1 error (X,Y,Z) on 1 qubit, which a subsequent round of error-correction (not implemented) is capable to resolve.
* **Note**: *Faults* arise in the circuit and might cause *errors* in the output state by propagation. We are considering 1- and 2-qubit faults in the circuit and know the Steane code is capable to correct a 1-qubit error (in output).
* For $|0_L\rangle$ states which are more than 1 error away from the closest stabilizer eigenstate, the flag qubit will be on, telling us to discard the state, as we cannot correct it.
    * In this case, the state would be interpreted as a $|1_L\rangle$ state with a single-qubit error, which will subsequently be corrected to $|1_L\rangle$, expl. `0b0000101`: smallest dist: to $K_1,K_2,K_3$ is 2 will be corrected to `0b0100101` which is a $|1_L\rangle$ (i.e. also a +1 eigenstate of all stabilizers).
    * Conversely, we can check if the inverse of an output state has a distance less or equal than 1 to one of the stabilizers. This is only the case for $|0_L\rangle$ with 2 or 3 qb. errors $\rightarrow$ $|1_L\rangle$ with 1 or 0 qb. errors, as both $|0_L\rangle$ and $|1_L\rangle$ are +1 eigenstates of all stabilizers.

In [11]:
# Build circuit
qc = pc.circuits.QuantumCircuit()
qc.append('init |0>',{0,4,5,6})
qc.append('init |+>',{1,2,3})
qc.append('CNOT',{(1,0),(3,5)})
qc.append('CNOT',{(2,6)})
qc.append('CNOT',{(1,4)})
qc.append('CNOT',{(2,0),(3,6)})
qc.append('CNOT',{(1,5)})
qc.append('CNOT',{(6,4)})
qc.append('CNOT',{(0,7)})
qc.append('CNOT',{(5,7)})
qc.append('CNOT',{(6,7)})
qc.append('measure Z', {0,1,2,3,4,5,6})

# All possible 1-qb and 2-qb faults
faults_1 = ['X', 'Y', 'Z']
faults_2 = list(itertools.product(faults_1 + ["I"], repeat=2)) # variation with repetition
faults_2.remove(('I','I')) # remove II as it is not a fault

def get_fcirc(tick, circuit_setup):
    fc = pc.error_gens.ErrorCircuits()
    qc = pc.circuits.QuantumCircuit(circuit_setup=circuit_setup)
    fc.add_circuits(time=tick, after_faults=qc)
    return fc

# Generate all possible faults at all possible positions in the circuit
gates_1 = [(i, locs) for i,(_,locs,_) in enumerate(qc.items()) if all(type(l) is int for l in locs)]
gates_2 = [(i, locs) for i,(_,locs,_) in enumerate(qc.items()) if all(type(l) is tuple for l in locs)]
f1_circs = [get_fcirc(t,[{f:{l}}]) for (t,ls) in gates_1 for l in ls for f in faults_1]
f2_circs = [get_fcirc(tick,[{fault:{loc}} for fault, loc in zip(ftup,ltup) if fault != 'I']) for tick,locs in gates_2 for ltup in locs for ftup in faults_2]

# CSS Stabilizer generators are pure (no X-Z-mixing)
# thus can be represented in simple binary form (111 instead of 111|000)
# Also each string is the +1 eigenvector in the corresponding basis P=Z or P=X.
# Ex. IIIZZZZ |xxx1111> = |xxx1111> and IIIZZZZ |xxx0000> = |xxx0000>
k1 = 0b0001111 # IIIPPPP
k2 = 0b1010101 # PIPIPIP
k3 = 0b0110011 # IPPIIPP
stab_gens = [k1,k2,k3] # generators of stabilizer group
stabs = [0,k1,k2,k3,k1^k2,k2^k3,k1^k3,k1^k2^k3] # stabs: powerset of generators, 0: identity

hamming2 = lambda a,b: bin(a^b).count('1') # Hamming dist: # of bit pairs with odd parity

n_qbs = 7 # no errors on flag qubit
circ_runner = pc.circuit_runners.Standard(seed=np.random.randint(1e9))
for f in f1_circs + f2_circs:
    
    # 1. Propagate errors through circuit & measure
    state = pc.simulators.PauliFaultProp(n_qbs)
    msmt, _ = circ_runner.run(state, qc, error_circuits=f) # updates 'state' to U|psi> & (non-destructive) msmt
    error = state.faults # error caused by propagated fault(s).
    
    # 2. Convert measurement to 7-bit string in Z-basis
    msmt_bit_str = BitArray(bin='0000000')
    for idx in list(*msmt.values()):
        msmt_bit_str[idx] = 1 # replace 0->1 at msmt index

    # 3. Check for logical error
    inv_msmt = msmt_bit_str.uint ^ 0b1111111 # |0_L>+n errs -> |1_L>+3-n errs
    c = [hamming2(inv_msmt, stab) for stab in stabs] # dist each stab to orth. msmt state 
    if min(c) <= 1: # orth. msmt state closer to stab than msmt state
        print(error) # Notice that in all cases there is an X on qb. 7 (flag)

{'X': {4, 6, 7}, 'Y': set(), 'Z': set()}
{'X': {4, 7}, 'Y': {6}, 'Z': {2, 3}}
{'X': {0, 2, 7}, 'Y': set(), 'Z': {3, 6}}
{'X': {0, 2, 7}, 'Y': set(), 'Z': set()}
{'X': {0, 7}, 'Y': {2}, 'Z': {3, 6}}
{'X': {0, 7}, 'Y': {2}, 'Z': set()}
{'X': {4, 6, 7}, 'Y': set(), 'Z': {2}}
{'X': {4, 7}, 'Y': {6}, 'Z': {2, 3}}
{'X': {4, 6, 7}, 'Y': set(), 'Z': set()}
{'X': {4, 7}, 'Y': {6}, 'Z': {3}}
{'X': {1, 5, 7}, 'Y': set(), 'Z': {4, 6}}
{'X': {1, 5, 7}, 'Y': set(), 'Z': set()}
{'X': {5, 7}, 'Y': {1}, 'Z': {4, 6}}
{'X': {5, 7}, 'Y': {1}, 'Z': set()}
{'X': {0, 2, 7}, 'Y': set(), 'Z': set()}
{'X': {2, 7}, 'Y': {0}, 'Z': set()}
{'X': {0, 7}, 'Y': {2}, 'Z': set()}
{'X': {7}, 'Y': {0, 2}, 'Z': set()}
{'X': {4, 6, 7}, 'Y': set(), 'Z': {3}}
{'X': {4, 7}, 'Y': {6}, 'Z': {3}}
{'X': {4, 6, 7}, 'Y': set(), 'Z': set()}
{'X': {4, 7}, 'Y': {6}, 'Z': set()}
{'X': {1, 5, 7}, 'Y': set(), 'Z': set()}
{'X': {1, 7}, 'Y': {5}, 'Z': set()}
{'X': {5, 7}, 'Y': {1}, 'Z': set()}
{'X': {7}, 'Y': {1, 5}, 'Z': set()}
{'X': {4, 6